1. iframe을 통해서 페이지를 표시하고 있다.
2. 요청할 페이지를 소스보기 하면 소스에 데이터가 있다.

In [4]:
import requests
import bs4
import time
import pandas

163608 수집 시작
1 / 5
2 / 5
3 / 5
4 / 5
5 / 5
132623 수집 시작
1 / 5
2 / 5
3 / 5
4 / 5
5 / 5
수집 완료


In [5]:
# 페이지를 요청하여 bs4객체를 반환하는 함수
def requestPage(url) :
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    return soup

In [6]:
# 필요한 정보를 가져온다.
def getData(soup) :
    # li 태그들을 가져온다.
    li_list = soup.select('div.score_result > ul > li')
    # print(li_list)
    
    reple_list = []
    score_list = []
    
    for li_tag in li_list :
        # 평점을 가져온다.
        star_score_tag = li_tag.select('div.star_score > em')
        star_score = star_score_tag[0].text.strip()
        # print(star_score_tag[0].prettify())
        # print(star_score)
        # 140자 평을 가져온다.
        score_reple_tag = li_tag.select('div.score_reple > p')
        score_reple = score_reple_tag[0].text.strip()
        # print(score_reple)
        
        #print(star_score)
        #print(score_reple)
        
        score_list.append(star_score)
        reple_list.append(score_reple)
        
    return score_list, reple_list
        
    

In [27]:
# 전체 페이지 개수를 반환하는 함수
def getPageCnt(movie_code) :
    # 접속할 페이지의 주소
    url = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={movie_code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1'
    # 요청한다.
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    # class가 score_total인 div를 가져온다.
    score_total_tag = soup.select('div.score_total > strong.total > em')[0]
    total_reple_cnt = score_total_tag.text.strip()
    # print(total_reple_cnt)
    # 계산한다.
    total_reple_cnt = total_reple_cnt.replace(',', '')
    total_page_cnt = int(total_reple_cnt) // 10
    if int(total_reple_cnt) % 10 > 0 :
        total_page_cnt += 1
        
    return total_page_cnt

In [32]:
# 영화 코드를 가져오는 함수
def getMovieCode() :
    # 영화 코드를 담을 리스트
    movie_code_list = []
    
    # 접속할 페이지의 주소
    url = 'https://movie.naver.com/movie/running/current.nhn?order=reserve'
    # 요청한다.
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    
    li_list = soup.select('ul.lst_detail_t1 > li')
    for li_tag in li_list :
        # a 태그를 가져온다.
        a_tag = li_tag.select('div.thumb > a')[0]
        # a 태그의 href 속성값을 가져온다.
        href = a_tag.attrs['href']
        # print(href)
        # 영화 코드 값을 가져온다.
        str1 = href.strip()
        str2 = str1.split('=')
        movie_code = str2[1]
        # print(movie_code)
        movie_code_list.append(movie_code)
        
    return movie_code_list
        
#getMovieCode()

In [ ]:
# 처음 저장인지 아는지를 구분하기 위한 변수
chk = False

# 가져올 영화 코드가 있는 리스트
# movie_list = [156464, 109906]
movie_list = getMovieCode()

# movie_list = movie_list[:2]

for movie_code in movie_list :
    
    print(f'{movie_code} 수집 시작')
    
    # 페이지 수를 가져온다.
    total_page_cnt = getPageCnt(movie_code)

    # total_page_cnt = 5


    for idx in range(total_page_cnt) :
        print(f'{idx + 1} / {total_page_cnt}')
        time.sleep(1)
        # 접속할 페이지의 주소
        url = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={movie_code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={idx + 1}'
        # 요청한다.
        soup = requestPage(url)
        # print(soup)
        # 데이터를 가져온다.
        score_list, reple_list = getData(soup)
        # print(score_list)
        # print(reple_list)
        # print('-----------------')
        # DataFrame을 생성한다.
        # 두 리스트를 합친다.
        total_list = zip(reple_list, score_list)
        df = pandas.DataFrame(list(total_list))

        if chk == False :
            chk = True
            df.columns = ['text', 'star']
            df.to_csv('naver_star_data.csv', index=False, encoding='utf-8-sig')
        else :
            df.to_csv('naver_star_data.csv', index=False, encoding='utf-8-sig', mode='a', header=False)

print('수집 완료')